# Unstructured API On-Demand Jobs Walkthrough

This walkthrough provides you with deep, hands-on experience with the [Unstructured API](https://docs.unstructured.io/api-reference/overview). As you follow along, you will learn how to use many of Unstructured's features for
[partitioning](https://docs.unstructured.io/ui/partitioning),
[enriching](https://docs.unstructured.io/ui/enriching/overview),
[chunking](https://docs.unstructured.io/ui/chunking),
and [embedding](https://docs.unstructured.io/ui/embedding).
These features are optimized for turning your source documents and data into information that is well-tuned for
[retrieval-augmented generation (RAG)](https://unstructured.io/blog/rag-whitepaper),
[agentic AI](https://unstructured.io/problems-we-solve#powering-agentic-ai),
and [model fine-tuning](https://www.geeksforgeeks.org/deep-learning/what-is-fine-tuning/).

This walkthrough uses two sample files to demonstrate how Unstructured identifies and processes content such as image, graphs, complex tables, non-English characters, handwriting, and poorly-scanned content. These files, which are available for you to download to your local machine, are as follows:

- Wang, Z., Liu, X., & Zhang, M. (2022, November 23). _Breaking the Representation Bottleneck of Chinese Characters: Neural Machine Translation with Stroke Sequence Modeling_. arXiv.org. https://arxiv.org/pdf/2211.12781. This 12-page PDF file features English and non-English characters, images, graphs, and complex tables. Throughout this walkthrough, this file's title is shortened to “Chinese characters” for brevity.
- United States Central Security Service. (2012, January 27). _National Cryptologic Museum Opens New Exhibit on Dr. John Nash_. United States National Security Agency. https://courses.csail.mit.edu/6.857/2012/files/H03-Cryptosystem-proposed-by-Nash.pdf. This PDF file features English handwriting and scanned images of documents. Throughout this walkthrough, this file's title is shortened to “Nash letters” for brevity.

If you are not able to complete any of the following steps, contact Unstructured Support at [support@unstructured.io](mailto:support@unstructured.io).

---
>
> 📝 _Note_
>
> This notebook uses Unstructured's _on-demand jobs_ functionality, which
> is designed to work *only with local files*.
>
> To process files (and data) in remote file and blob storage, databases, and
> vector stores, you must use other workflow operations in the Unstructured
> API. To learn how, see for example the notebook
>[Dropbox-To-Pinecone Connector API Quickstart for Unstructured](https://colab.research.google.com/github/Unstructured-IO/notebooks/blob/main/notebooks/Dropbox_To_Pinecone_Connector_Quickstart.ipynb).  
>
---
>
> 💡 _What's this?_
>
> As you move through this walkthrough, you will notice tips like this one.
> These tips are designed to help expand your knowledge about Unstructured as
> you go. Feel free to skip these tips for now if you are in a hurry. You can
> always return to them later to learn more.
>
---

## Requirements

To run this notebook, you will need:

- An Unstructured account. To sign up for an account, go to https://unstructured.io/?modal=try-for-free. After you sign up, you are immediately signed in to your new Unstructured **Let's Go** account, at https://platform.unstructured.io.
- An Unstructured API key, as follows:

  1. After you are signed in to your account, on the sidebar click **API Keys**.
  2. Click **Generate New Key**.
  3. Enter some meaningful display name for the key, and then click **Continue**.
  4. Next to the new key's name, click the **Copy** icon. The key's value is copied to your system's clipboard. If you lose this key, simply return to the list and click **Copy** again.

- One or more local files for Unstructured to process. This notebook assumes that you are using the "Chinese characters" and "Nash letters" PDF files, and that these PDFs are in a subfolder that is accessible from this notebook. The easiest and fastest way to create this subfolder is as follows:

  1. On this notebook's sidebar, click the folder (**Files**) icon. By default, the **Files** pane will show all files and subfolders within the `/content` folder. (This folder also contains a hidden `.config` subfolder and a `sample_data` subfolder.)
  2. Right-click in a blank area anywhere below the existing list of subfolders and files in the **Files** pane, and then click **New folder**.
  3. Enter a name for the new subfolder within `/content`. This notebook assumes the subfolder is named `input`.
  4. To upload files to this subfolder, do the following:

     a. Rest your mouse pointer on the `input` subfolder.<br/>
     b. Click the ellipsis (three dots) icon, and then click **Upload**.<br/>
     c. Browse to and select the "Chinese characters" file on your local machine that you want to upload to this `input` subfolder.<br/>
     d. Repeat this process to upload the "Nash letters" file to this `input` subfolder.<br/>

---
>
> ⚠️ **Important**: Each on-demand job is limited to 10 files, and each file is limited to 10 MB in size.
>
> If you need to launch a series of on-demand jobs in rapid succession, you must wait at least once second between launch requests. Otherwise, you will receive a rate limit error.
>
> A maximum of 5 on-demand jobs can be running in your Unstructured account. If you launch a new on-demand job but 5 existing on-demand jobs are still running, the new on-demand job will remain in a scheduled state until one of the 5 existing on-demand jobs is done running.
>
---

- A destination subfolder for Unstructured to send its processed results to. This notebook assumes that the destination folder is accessible from this notebook. The easiest and fastest way to create this folder is as follows:

  1. If the **Files** pane is not already open, then on this notebook's sidebar, click the folder (**Files**) icon. By default, the **Files** pane will show all files and subfolders within the `/content` folder.
  2. Right-click in a blank area anywhere below the existing list of subfolders and files in the **Files** pane, and then click **New folder**.
  3. Enter a name for the new subfolder within `/content`. This notebook assumes the subfolder is named `output`.

---
>
> ⚠️ **Warning**: Any files that you upload to these `input` or `output`
> subfolders will be deleted whenever Google Colab disconnects or resets, for
> example due to inactivity, manual restart, or session timeout.
>
---

## Step 1: Run a basic on-demand job

In this step, you run an [on-demand job](https://docs.unstructured.io/api-reference/workflow/overview#run-an-on-demand-job) in your Unstructured account. This on-demand job contains a basic [workflow](https://docs.unstructured.io/api-reference/workflow/workflows). The workflow lasts only for the duration of this on-demand job.

Workflows are defined sequences of processes that automate the flow of data from your source documents and data into Unstructured for processing. Unstructured then sends its processed data over into your destination file storage locations, databases, and vector stores. Your RAG apps, agents, and models can then use this processed data in those destinations to do things more quickly and accurately such as
[answering users' questions](https://learn.microsoft.com/en-us/azure/developer/ai/advanced-retrieval-augmented-generation),
[automating business processes](https://unstructured.io/problems-we-solve#business-process-automation),
and [expanding your organization's available body of knowledge](http://knowledgemanagement.ie/the-critical-role-of-knowledge-management-as-a-foundation-for-llms-and-ai/).

In this notebook, your source documents are local, and Unstructured stores its processed data locally as well. However, as just described, Unstructured can work with remote source documents (and data), too.

---
>
> 💡 _Which kinds of remote sources and destinations does Unstructured support?_
>
> Unstructured can connect to many types of remote sources and destinations including
> file storage services such as Amazon S3 and Google Cloud Storage; databases
> such as PostgreSQL; and vector storage and database services such as MongoDB
> Atlas and Pinecone.
>
> See the full list of [supported source connectors](https://docs.unstructured.io/api-reference/workflow/sources/overview) and [supported destination connectors](https://docs.unstructured.io/api-reference/workflow/destinations/overview).
>
---
>
> 💡 _Which kinds of files does Unstructured support?_
>
> Unstructured can process a wide variety of file types including PDFs, word
> processing documents, spreadsheets, slide decks, HTML, image files, emails,
> and more.
>
> See the full list of [supported file types](https://docs.unstructured.io/api-reference/supported-file-types).
>
---

1. In the following cell, replace `<your-unstructured-api-key>` with your pasted Unstructured API key's value and then run the cell, which sets the constant `UNSTRUCTURED_API_KEY` to your API key's value.

In [ ]:
UNSTRUCTURED_API_KEY = "<your-unstructured-api-key>"

2. Run the following cell to install the Unstructured Python SDK on a virtual machine (VM) in Google's cloud. This VM is associated with this notebook.

In [ ]:
!pip install unstructured-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 14.5 MB/s eta 0:00:00


3. Run the following cell, which sets the notebook's local input and output subfolders.

In [ ]:
INPUT_PATH = "/content/input"
OUTPUT_PATH = "/content/output"

4. Run the following cell, which runs a basic on-demand job. This job contains an Unstructured custom workflow with a single stage or step (known as a workflow _node_) for partitioning. This workflow uses the **VLM** [partitioning](https://docs.unstructured.io/ui/partitioning) strategy to turn the contents of your documents and semi-structured data into a data format that is fine-tuned for well-tuned for retrieval-augmented generation (RAG), agentic AI, and model fine-tuning.

   The results of running this cell include the on-demand job's ID and a list of job outputs. This list's items include a file ID and an output node ID for each file that Unstructured processes. You will use these file IDs and output node IDs later to retrieve Unstructured's processed file data.

In [ ]:
from unstructured_client import UnstructuredClient
from unstructured_client.models.operations import CreateJobRequest
from unstructured_client.models.shared import BodyCreateJob, InputFiles
import os, json

def run_on_demand_job(client, input_dir, template_id=None, job_nodes=None):
    request_data = {}
    files = []

    for filename in os.listdir(input_dir):
        full_path = os.path.join(input_dir, filename)

        # Skip non-files (for example, directories).
        if not os.path.isfile(full_path):
            continue

        files.append(
            (
                InputFiles(
                    content=open(full_path, "rb"),
                    file_name=filename,
                    content_type="application/pdf"
                )
            )
        )

    if template_id is not None:
        request_data = json.dumps({"template_id": template_id})
    elif job_nodes is not None:
        request_data = json.dumps({"job_nodes": job_nodes})
    else:
        raise ValueError(f"Must provide a workflow template ID or a custom workflow definition for this job (but not both).")

    # Run the on-demand job, capturing the job ID and the job's
    # input/output file IDs and output node IDs.
    response = client.jobs.create_job(
        request=CreateJobRequest(
            body_create_job=BodyCreateJob(
                request_data=request_data,
                input_files=files
            )
        )
    )

    job_id = response.job_information.id
    job_input_file_ids = response.job_information.input_file_ids
    job_output_node_files = response.job_information.output_node_files

    return job_id, job_input_file_ids, job_output_node_files

vlm_partitioner_node = {
    "name": "Partitioner",
    "subtype": "vlm",
    "type": "partition",
    "settings": {
        "provider": "vertexai",
        "model": "gemini-2.0-flash-001",
        "is_dynamic": False,
        "allow_fast": True
    }
}

job_nodes = [ vlm_partitioner_node ]

job_id = ""
job_input_file_ids = []
job_output_node_files = []

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

Job ID: 1f91e440-eeb2-407d-a4e8-9131693e392d

Input file details:

221112781v1-3b190ed1.pdf
H03-Cryptosystem-proposed-by-Nash-3edb6a01.pdf

Output node file details:

file_id='221112781v1-3b190ed1.pdf' node_id='28a283ce-fd39-4bfa-8e4a-7cb847856853' node_subtype='vlm' node_type='partition'
file_id='H03-Cryptosystem-proposed-by-Nash-3edb6a01.pdf' node_id='28a283ce-fd39-4bfa-8e4a-7cb847856853' node_subtype='vlm' node_type='partition'


---
>
> 💡 _What other kinds of settings can I set for my workflow?_
>
> Your workflow can be set to run automatically (on a regular time schedule).
> You can also set your workflow to use a paritioning strategy such as
> **Auto**, **Fast**, or **High Res**, instead of **VLM**.
> [Learn how](https://docs.unstructured.io/api-reference/workflow/workflows).
>
---
>
> 💡 _Why am I specifying this particular vision language model here? When would I choose one of the available models over another?_
>
> A _vision language model_ (VLM) is designed to use sophisticated AI
> techniques and logic to combine advanced image and text understanding,
> resulting in more accurate and contextually-rich output.
>
> The following code uses the Gemini Flash 2.0 VLM offered by Vertex AI.
> As VLMs are constantly being released
> and improved, Unstructured is always
> adding to and updating its list of supported VLMs. If you aren't getting
> consistent results with one VLM for a particular set of files, switching over
> to another one might improve your results, depending on that VLM's
> capabilities and the sample data that is was trained on.
>
---
>
> 💡 _When would I choose **Auto**, **Fast**, **High Res**, or **VLM**?_
>
> - **Auto** is recommended in most cases. It lets Unstructured figure out the
> best strategy to switch over to for each incoming file (and even for each
> page if the incoming file is a PDF), so you don't have to!
> - **Fast** is only for when you know for certain that none of your files have
> tables, images, or multilanguage, scanned, or handwritten content in them.
> It's optimized for partitioning text-only content and is the fastest of all
> the strategies. It can recognize the text for only a few languages other than
> English.
> - **High Res** is only for when you know for certain that at least one of your
> files has images or simple tables in them, and that none of your files also
> have scanned or handwritten content in them. It can recognize the text for
> more languages than **Fast** but not as many as **VLM**.
> - **VLM** is great for any file, but it is best when you know for certain that
> some of your files have a combination of tables (especially complex ones),
> images, and multilanguage, scanned, or handwritten content. It's the highest
> quality but slowest of all the strategies.
>
> In this walkthrough, you use the **VLM** strategy
> only to see how each of these strategies works with a combination of complex
> tables, images, and multilanguage, scanned, and handwritten content. In
> practice, for these kinds of files you would likely just want to choose
> **Auto**.
>
---
>
> 💡 _This workflow has only a partitioner. What about enriching, chunking, and embedding?_
>
> Don't worrry; you will add nodes to this workflow for enriching, chunking,
> and embedding later in this notebook.
>
---

5. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

   - If the job's final status is completed but with errors, or is failed, continue ahead to step 6 in this procedure to try to find out why.
   - If the job's final status is stopped, try running the previous cell in step 4 again to produce a new job for this workflow.
   - If the job's final status is successfully completed, skip ahead to step 7 in this procedure.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#get-processing-details-for-a-job).

In [ ]:
import time
from unstructured_client import UnstructuredClient

def poll_job_status(client, job_id):
    while True:
        response = client.jobs.get_job(
            request={
                "job_id": job_id
            }
        )

        job = response.job_information

        if job.status == "SCHEDULED":
            print("Job is scheduled, polling again in 10 seconds...")
            time.sleep(10)
        elif job.status == "IN_PROGRESS":
            print("Job is in progress, polling again in 10 seconds...")
            time.sleep(10)
        else:
            print("Job is completed.")
            break

    return job


with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is completed.
Job details:
---
{
    "created_at": "2025-12-10T20:10:20.316640Z",
    "id": "1f91e440-eeb2-407d-a4e8-9131693e392d",
    "status": "COMPLETED",
    "workflow_id": "760b3ad5-b747-47ed-b882-b84534c7d9b2",
    "workflow

6. If the job is completed but with errors, or is failed, run the following cell to try to find out why. Otherwise, skip ahead to step 7 in this procedure.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#get-failed-file-details-for-a-job).

In [ ]:
from unstructured_client.models.operations import GetJobFailedFilesRequest

def get_failed_files_details(client, job_id):
    response = client.jobs.get_job_failed_files(
        request=GetJobFailedFilesRequest(
            job_id=job_id
        )
    )

    info = response.job_failed_files

    if info.failed_files.__len__() > 0:
        print(f"{info.failed_files.__len__()} failed file(s):")

        for failed_file in info.failed_files:
            print(f"---")
            print(f"document: {failed_file.document}")
            print(f"error:    {failed_file.error}")
    else:
        print(f"No failed files.")

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    get_failed_files_details(client, job_id)

No failed files.


7. If the job is successfully completed, run the following cell to download the data that Unstructured generated for the "Chinese characters" file to this notebook's local `/content/output` subfolder.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#download-a-processed-local-file-from-a-job).

In [ ]:
from unstructured_client.models.operations import DownloadJobOutputRequest

def download_job_output(client, job_id, job_input_file_ids, output_dir):
    for job_input_file_id in job_input_file_ids:
        print(f"Attempting to get processed results from file_id '{job_input_file_id}'...")

        response = client.jobs.download_job_output(
            request=DownloadJobOutputRequest(
                job_id=job_id,
                file_id=job_input_file_id
            )
        )

        output_path = os.path.join(output_dir, f"{job_input_file_id}.json")

        with open(output_path, "w") as f:
            json.dump(response.any, f, indent=4)

        print(f"Saved output for file_id '{job_input_file_id}' to '{output_path}'.\n")

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

Attempting to get processed results from file_id '221112781v1-3b190ed1.pdf'...
Saved output for file_id '221112781v1-3b190ed1.pdf' to '/content/output/221112781v1-3b190ed1.pdf.json'.

Attempting to get processed results from file_id 'H03-Cryptosystem-proposed-by-Nash-3edb6a01.pdf'...
Saved output for file_id 'H03-Cryptosystem-proposed-by-Nash-3edb6a01.pdf' to '/content/output/H03-Cryptosystem-proposed-by-Nash-3edb6a01.pdf.json'.



8. You can now look at the data that Unstructured generated. Do this as follows:

   a. On this notebook's sidebar, click the folder (**Files**) icon, if the **Files** pane is not already shown.<br/>
   b. In the **Files** pane, click to expand the `output` folder.<br/>
   c. Double-click the file titled `221112781v1-<some_number>.pdf.json`, where `221112781v1-<some_number>.pdf` matches the related `file_id` value above.<br/>
   d. The file's contents appear in a pane on the right side of this notebook.

---
>
> 💡 _What am I looking at in the output here?_
>
> - Unstructured outputs its results in industry-standard [JSON](https://www.json.org/) format, which is
> ideal for RAG, agentic AI, and model fine-tuning.
> - Each object in the JSON is called a [document element](https://docs.unstructured.io/ui/document-elements) and contains a `text`
> representation of the content that Unstructured detected for the particular
> portion of the document that was analyzed.
> - The `type` is the kind of document element that Unstructured categorizes it
> as, such as whether it is a title (`Title`), a table (`Table`), an image
> (`Image`), a series of well-formulated sentences (`NarrativeText`), some kind
> of free text (`UncategorizedText`), a part of a list (`ListItem`), and so on. [Learn more](https://docs.unstructured.io/ui/document-elements#element-type).
> - The `element_id` is a unique identifier that Unstructured generates to
> refer to each document element. [Learn more](https://docs.unstructured.io/ui/document-elements#element-id).
> - `metadata` contains supporting details about each document element, such as
> the page number it occurred on, the file it occurred in, and so on. [Learn more](https://docs.unstructured.io/ui/document-elements#metadata).
>
---

9. Some interesting portions of the output include the following. To search for text in the output, right-click anywhere inside the output pane, click **Command Palette**, enter or click **Find**, and then enter the text you want to search for:

   - The Chinese characters on page 3. Search for the text `verbs. The characters`. Notice how the Chinese characters are interpreted as their Unicode equivalents. For example, `\u624e` is 扎.
   - The tables on pages 1, 3, 6, 7-9, and 12. Search for the text `"Table"` (including the quotation marks). Notice especially the `text` and `metadata.text_as_html` output for each of these table elements. To quickly jump among each of the tables in this output, click the **Next Match (Enter)** (down arrow) icon in the **Find** bar.
   - The images on pages 3, 7, and 8. Search for the text `"Image"` (including the quotation marks). Notice especially the `text` and `metadata.text_as_html` output for each of these image elements. To quickly jump among each of the images in this output, click the **Next Match (Enter)** (down arrow) icon in the **Find** bar.

10. Now open the `H03-Cryptosystem-proposed-by-Nash-<some_number>.pdf.json` file, where `H03-Cryptosystem-proposed-by-Nash-<some_number>.pdf` matches the related `file_id` value above.
11. Some interesting portions of the output include the following:

    - The handwriting on page 3. Search for the text `Dear Major Grosjean`.
    - The mimeograph on page 18. Search for the text `The system which`.

## Step 2: Experiment with enriching

In this step, you add several [enrichments](https://docs.unstructured.io/ui/enriching/overview) to your workflow, such as generating summary descriptions of detected images and tables, HTML representations of detected tables, detected entities (such as people and organizations) and the inferred relationships among these entities, and improving the accuracy of initially-detected complex text.

---
>
> 💡 _Can you tell me more about what each of these enrichments actually do?_
>
> The _image description_ enrichment generates a summary description of each
> detected image. This can help you to more quickly and easily understand what
> each image is all about without having to stop to manually visualize and
> interpret the image's content yourself. This also provides additional helpful
> context about the image for your RAG apps, agents, and models.
> [Learn more](https://docs.unstructured.io/ui/enriching/image-descriptions).
>
> The _table description_ enrichment generates a summary description of each
> detected table. This can help you to more quickly and easily understand what
> each table is all about without having to stop to manually read through the
> table's content yourself. This also provides additional helpful context about
> the table for your RAG apps, agents, and models.
> [Learn more](https://docs.unstructured.io/ui/enriching/table-descriptions).
>
> The _table-to-HTML_ enrichment generates an HTML representation of each
> detected table. This can help you to more quickly and accurately recreate the
> table's content elsewhere later as needed. This also provides additional
> context about the table's structure for your RAG apps, agents, and models.
> [Learn more](https://docs.unstructured.io/ui/enriching/table-to-html).
>
> The _named entity recognition (NER)_ enrichment generates a list of detected
> entities (such as people and organizations) and the inferred relationships
> among these entities. This provides additional context about these entities'
> types and their relationships for your graph databases, RAG apps, agents, and
> models.
> [Learn more](https://docs.unstructured.io/ui/enriching/ner).
>
> The _generative optical character recognition (generative OCR)_ enrichment
> improves the accuracy of initially-detected complex text, such as text that
> is presented in non-standard orientations, watermarked text, or text with a
> mixture of international characters.
>[Learn more](https://docs.unstructured.io/ui/enriching/generative-ocr).
>
---

1. Run the following cell, which adds workflow nodes for generating the image and table summary descriptions, table HTML, entities and their relationships, and generative OCR.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/workflows#enrichment-node).

In [ ]:
image_description_enrichment_node = {
    "name": "Anthropic Image Description",
    "subtype": "anthropic_image_description",
    "type": "prompter",
    "settings": {}
}

table_description_enrichment_node = {
    "name": "Anthropic Table Description",
    "subtype": "anthropic_table_description",
    "type": "prompter",
    "settings": {}
}

table_to_html_enrichment_node = {
    "name": "Anthropic Table to HTML",
    "subtype": "anthropic_table2html",
    "type": "prompter",
    "settings": {}
}

named_entity_recognition_enrichment_node = {
    "name": "Anthropic NER",
    "subtype": "anthropic_ner",
    "type": "prompter",
    "settings": {}
}

generative_ocr_enrichment_node = {
    "name": "Anthropic Generative OCR",
    "subtype": "anthropic_ocr",
    "type": "prompter",
    "settings": {}
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node
]

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

2. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

3. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
def clear_output_dir_files(output_dir):
  for name in os.listdir(output_dir):
    file_path = os.path.join(output_dir, name)
    if os.path.isfile(file_path): # Skip any subfolders.
        os.remove(file_path)


with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

4. Some interesting portions of the output, for example in the "Chinese characters" file, include the following:

   - The tables on pages 1, 3, 6, 7-9, and 12. Search for the text `"Table"` (including the quotation marks). Notice the summary description for each of these tables in these table elements' `text` fields. Also notice the `text_as_html` field for each of these tables. Remember, to quickly jump among each of the tables in this output, click the **Next Match (Enter)** (down arrow) icon in the **Find** bar.
   - The images on pages 3, 7, and 8. Search for the text `"Image"` (including the quotation marks). Notice the summary description for each of these image elements' `text` fields. Remember, to quickly jump among each of the images  in this output, click the **Next Match (Enter)** (down arrow) icon in the **Find** bar.
   - The identified entities and inferred relationships among them. Search for the text `Zhijun Wang`. Of the eight instances of this name, notice the author's identification as a `PERSON` three times, the author's `published` relationship twice, and the author's `affiliated_with` relationship twice. Remember, to quickly jump among each of the instances of `Zhijun Wang` in this output, click the **Next Match (Enter)** (down arrow) icon in the **Find** bar.

## Step 3: Experiment with chunking

In this step, you apply [chunking](https://docs.unstructured.io/ui/chunking) to your workflow. Chunking is the process where Unstructured rearranges the resulting document elements' `text` content into manageable "chunks" to stay within the limits of an AI model and to improve retrieval precision.

---
>
> 💡 _What kind of chunking strategy should I use, and how big should my chunks be?_
>
> Unfortunately, there is no one-size-fits-all answer to this question.
> However, there are some general considerations and guidelines that can help
> you to determine the best chunking strategy and chunk size for your specific
> use case. Be sure of course to also consult the documentation for your target
> AI model and downstream application toolsets.
>
> Is your content primarily organized by title, by page, by interrelated
> subject matter, or none of these? This can help you determine whether a
> by-title, by-page, by-similarity, or basic (by-character) chunking strategy
> is best. (You'll experiment with each of these strategies here later.)
>
> If your chunks are too small, they might lose necessary context, leading to
> the model providing inaccurate, irrelevant, or hallucinated results. On the
> other hand, if your chunks are too large, the model can struggle with the
> sheer volume of information, leading to information overload, diluted
> meaning, and potentially higher processing costs. You should aim to find a
> balance between chunks that are big enough to contain meaningful information,
> while small enough to enable performant applications and low latency
> responses.
>
> For example, smaller chunks of 128 or 256 tokens might be sufficient for
> capturing more granular semantic information, while larger chunks of 512 or
> 1024 tokens might be better for retaining more context. It's important here
> to note that _tokens_ and _characters_ are not the same thing! In terms of
> characters, for English text, a common approximation is 1 token being equal
> to about 3 or 4 characters or three-quarters of a word. Many AI model
> providers publish their own token-to-character calculators online that you
> can use for estimation purposes.
>
> You should experiement with a variety of chunk sizes, taking into account the
> kinds of content, the length and complexity of user queries and agent tasks,
> the intended end use, and of course the limits of the models you are using.
> Try different chunking strategies and sizes with your models and evaluate the
> results for yourself.
>
---

1. Run the following code, which changes the workflow to add a workflow node that chunks the document elements' `text` by using a character-based strategy.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/workflows#chunker-node).

---
>
> 💡 _What kinds of chunking settings are available, and what does each setting do?_
>
> - Contextual chunking (`contextual_chunking_strategy`) prepends
>   chunk-specific explanatory context to each chunk, which has been shown to
>   yield significant improvements in downstream retrieval accuracy.
>   [Learn more](https://docs.unstructured.io/ui/chunking#contextual-chunking).
> - Include original elements (`include_orig_elements`) outputs into each
>   chunk's `metadata` field's `orig_elements` value the elements that were
>   used to form that particular chunk.
>   [Learn more](https://docs.unstructured.io/ui/chunking#include-original-elements-setting).
> - Max characters (`max_characters`) is the "hard" or maximum number of
>   characters that any one chunk can contain. Unstructured cannot exceed this
>   number when forming chunks.
>   [Learn more](https://docs.unstructured.io/ui/chunking#max-characters-setting).
> - New after n characters (`new_after_n_characters`) is the "soft" or
>   approximate number of characters that any one chunk can contain.
>   Unstructured can exceed this number if needed when forming chunks (but
>   still cannot exceed the max characters setting).
>   [Learn more](https://docs.unstructured.io/ui/chunking#new-after-n-characters-setting).
> - Overlap (`overlap`), when applied (see `overlap_all`), prepends to the
>   current chunk the specified number of characters from the previous chunk,
>   which can help provide additional context about this chunk relative to the
>   previous chunk.
>   [Learn more](https://docs.unstructured.io/ui/chunking#overlap-setting).
> - Overlap all (`overlap_all`), applies the `overlap` setting (if greater than
>   zero) to all chunks. Otherwise, setting `overlap_all` to `False` that the
>   `overlap` setting (if greater than zero) is applied only in edge cases
>   where `normal` chunks cannot be formed by combining whole elements. Set
>   `overlap_all` to `True` with caution as it can introduce noise into
>   otherwise clean semantic units.
>   [Learn more](https://docs.unstructured.io/ui/chunking#overlap-all-setting).
> - Combine text under n characters (`combine_text_under_n_characters`)
>   combines elements from a section into a chunk until a section reaches a
>   length of this many characters.
>   [Learn more](https://docs.unstructured.io/ui/chunking#combine-text-under-n-characters-setting).
> - Multipage sections (`multipage_sections`), when set to `True`, allows
>   sections to span multiple pages.
>   [Learn more](https://docs.unstructured.io/ui/chunking#multipage-sections-setting).
> - The similarity threshold (`similarity_threshold`) is a number between `0`
>   and `1` exclusive (`0.01` to `0.99` inclusive). `0.01` means that any two
>   segments of text that are being compared to each other and are considered
>   least identical in semantic meaning to each other are more likely to be
>   combined into the same chunk together, when such combining must occur.
>   `0.99` means that any two segments of text that are being compared to each
>   other and are considered almost identical in semantic meaning to each other
>   are more likely to be combined into the same chunk together, when such
>   combining must occur. Numbers toward `0.01` bias toward least-identical
>   semantic matches, while numbers toward `0.99` bias toward near-identical
>   semantic matches.
>   [Learn more](https://docs.unstructured.io/ui/chunking#similarity-threshold-setting).

In [ ]:
chunk_by_character_node = {
    "name": "Chunker",
    "subtype": "chunk_by_character",
    "type": "chunk",
    "settings": {
        "include_orig_elements": True,
        "new_after_n_chars": 400,
        "max_characters": 500,
        "overlap": 50
    }
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node,
    chunk_by_character_node
]

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

2. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

3. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

4. Open the new version of the output files. To explore the chunker's results, search for the text `"CompositeElement"` (with quotation marks).
6. (Steps 5-8 are optional) Now run the following cell, which changes the workflow's chunker node to chunk the document elements' text by using a title-based strategy.

In [ ]:
chunk_by_title_node = {
    "name": "Chunker",
    "subtype": "chunk_by_title",
    "type": "chunk",
    "settings": {
        "include_orig_elements": True,
        "new_after_n_chars": 400,
        "max_characters": 500,
        "overlap": 50
    }
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node,
    chunk_by_title_node
]

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

6. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

7. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

8. Open the new version of the output files. To explore the chunker's results, search for the text "CompositeElement" (with quotation marks). Notice that the lengths of some of the chunks that immediately precede titles might be shortened due to the presence of the title impacting the chunk's size.
9. (Steps 9-12 are optional) Now run the following cell, which changes the workflow's chunker node to chunk the document elements' text by using a page-based strategy.

In [ ]:
chunk_by_page_node = {
    "name": "Chunker",
    "subtype": "chunk_by_page",
    "type": "chunk",
    "settings": {
        "include_orig_elements": True,
        "new_after_n_chars": 400,
        "max_characters": 500,
        "overlap": 50
    }
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node,
    chunk_by_page_node
]

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

10. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

11. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

12. Open the new version of the output files. To explore the chunker's results, search for the text "CompositeElement" (with quotation marks). Notice that the lengths of some of the chunks that immediately precede page breaks might be shortened due to the presence of the page break impacting the chunk's size.
13. (Steps 13-20 are optional) Now run the following cell, which changes the workflow's chunker node to chunk the document elements' text by using a similarity-based strategy.

In [ ]:
chunk_by_similarity_node = {
    "name": "Chunker",
    "subtype": "chunk_by_similarity",
    "type": "chunk",
    "settings": {
        "similarity_threshold": 0.99,
        "include_orig_elements": True,
        "max_characters": 500
    }
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node,
    chunk_by_similarity_node
]

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

14. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

15. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

16. Open the new version of the output files. To explore the chunker's results, search for the text `"CompositeElement"` (with quotation marks). Notice that the lengths of many of the chunks fall well short of the `max_characters` limit. This is because a similarity threshold of `0.99` means that only sentences or text segments with a near-perfect semantic match will be grouped together into the same chunk. This is an extremely high threshold, resulting in very short, highly specific chunks of text.
17. Now run the following cell, which changes the existing workflow's chunker node to use a different similarity threshold.

In [ ]:
chunk_by_similarity_node = {
    "name": "Chunker",
    "subtype": "chunk_by_similarity",
    "type": "chunk",
    "settings": {
        "similarity_threshold": 0.01,
        "include_orig_elements": True,
        "max_characters": 500
    }
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node,
    chunk_by_similarity_node
]

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

18. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

19. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

20. Open the new version of the output files. To explore the chunker's results, search for the text `"CompositeElement"` (with quotation marks). Notice now that many of the chunks will now come closer to the `max_characters` limit. This is because a similarity threshold of `0.01` provides an extreme tolerance of differences between pieces of text, grouping almost anything together.

## Step 4 (Optional): Experiment with embedding

In this step, you generate [embeddings](https://docs.unstructured.io/ui/embedding) for your workflow. Embeddings are vectors of numbers that represent various aspects of the text that is extracted by Unstructured. These vectors are stored or "embedded" next to the text itself in a vector store or vector database. Chatbots, agents, and other AI solutions can use these vector embeddings to more efficiently and effectively find, analyze, and use the associated text. These vector embeddings are generated by an embedding model that is provided by an embedding provider. For the best embedding model to apply to your use case, see the documentation for your target downstream application toolsets.

1. Run the following cell, which changes the workflow to add a workflow node that uses the `text-embedding-3-small` embedding model provided by Azure OpenAI.

In [ ]:
embedder_node = {
    "name": "Embedder",
    "subtype": "azure_openai",
    "type": "embed",
    "settings": {
        "model_name": "text-embedding-3-small"
    }
}

job_nodes = [
    vlm_partitioner_node,
    image_description_enrichment_node,
    table_description_enrichment_node,
    table_to_html_enrichment_node,
    named_entity_recognition_enrichment_node,
    generative_ocr_enrichment_node,
    chunk_by_character_node,
    embedder_node
]

job_id, output_node_files = run_on_demand_job(
    input_dir = INPUT_PATH,
    job_nodes = job_nodes
)

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job_id, job_input_file_ids, job_output_node_files = run_on_demand_job(
        client = client,
        input_dir = INPUT_PATH,
        job_nodes = job_nodes
    )

    print(f"Job ID: {job_id}\n")
    print("Input file details:\n")

    for job_input_file_id in job_input_file_ids:
        print(job_input_file_id)

    print("\nOutput node file details:\n")

    for output_node_file in job_output_node_files:
        print(output_node_file)

2. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete. This job polling could take a few minutes or more.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    job = poll_job_status(client, job_id)
    print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

3. If the job is successfully completed, run the following cell to clear out the downloaded data from the previous on-demand job, and then download the data that Unstructured generated for the "Chinese characters" and "Nash letters" files to this notebook's local `/content/output` subfolder.

In [ ]:
with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    clear_output_dir_files(OUTPUT_PATH)
    download_job_output(client, job_id, job_input_file_ids, OUTPUT_PATH)

4. To explore the embeddings, open the new version of the output files, and search for the text `"embeddings"`.

---
>
> 💡 _What do all of these numbers mean?_
>
> All by themselves, the numbers in the `embeddings` field of the output have
> no human-interpretable meaning on their own. However, when combined with the
> specific text that these numbers are associated with, and the embedding
> model's logic that was used to generate these numbers, the numbers in the
> `embeddings` field are extremely powerful when leveraged by downstream
> chatbots, agents, and other AI solutions.
>
> These numbers typically represent complex, abstract attributes about the text
> that are known only to the embedding model that generated these numbers.
> These attributes can be about the text's overall sentiment, intent, subject,
> semantic meaning, grammatical function, relationships between words, or any
> number of other things that the model is good at figuring out. This is why
> the embedding model you choose here must be the exact same embedding model
> that you use in any related chatbot, agent, or other AI solution that relies
> on these numbers. Otherwise, the numbers that are generated here will not
> have the same meaning downstream as well. Also, the number of dimensions (or
> the number of numbers in the `embeddings` field) you choose here must also be
> the exact same number of dimensions downstream as well.
>
> To repeat, the name and number of dimensions for the embedding model you
> choose here must be the exact same name and number of dimensions for the
> embedding model you use in your related downstream chatbots, agents, and
> other AI solutions that rely on this particular text and its associated
> embeddings that were generated here.

## Next steps

Congratulations! You are now able to run Unstructured on-demand jobs that that partition, enrich, chunk, and embed your local source documents, producing context-rich data that is ready for retrieval-augmented generation (RAG), agentic AI, and model fine-tuning.

Right now, your workflow only accepts local files for input. Your workflow also only sends Unstructured's processed data to be saved locally as a JSON file. You can create workflows that accept files and data from&mdash;and send Unstructured's processed data to&mdash;one or more remote file storage locations, databases, and vector stores. To learn how to do this, try the [Dropbox-to-Pinecone Connector API Quickstart for Unstructured](https://colab.research.google.com/github/Unstructured-IO/notebooks/blob/main/notebooks/Dropbox_To_Pinecone_Connector_Quickstart.ipynb) notebook.

Unstructured also offers a user interface (UI), which allows you to use a graphical user interface to work with Unstructured instead of only with the API. For details, see the [Unstructured UI Overview](https://docs.unstructured.io/ui/overview).

If you are not able to complete any of the preceding quickstarts, contact Unstructured Support at [support@unstructured.io](mailto:support@unstructured.io).